依存パッケージのインストール  




In [12]:
!pip3 install category_encoders pandas sklearn seaborn pickle5 

/content


In [65]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import category_encoders as ce
import pandas as pd
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
import pickle5 as pikle

import seaborn as sns

データセットの読み込み  
今回はCryptoKittiesの全取引履歴と猫の属性をcsv化したものを使用する。  

容量143M  
レコード数819453件(かなりの重複込み)

In [46]:
# データセットの読み込み
df_trade = pd.read_csv("./data/tradeData.csv")

# カラム一覧の表示
print(df_trade.columns)

# レコード数の表示
len(df_trade)

Index(['blockNumber', 'value', 'tokenId', 'txHash', 'generation', 'body',
       'coloreyes', 'eyes', 'pattern', 'mouth', 'colorprimary',
       'colorsecondary', 'colortertiary', 'base_colour', 'fur', 'wild_element',
       'kitty_type'],
      dtype='object')


819452

前処理

In [47]:
# 重複行カウント
print(df_trade.duplicated().value_counts())

#重複行削除
df_trade = df_trade.drop_duplicates(keep='last')

# 有効データ件数
print(len(df_trade))

# 欠損値の置換
df_trade = df_trade.fillna(0)

# value == 0以下を除外
# 価格が0円なのは、タダであげたか、新しく猫が生まれたかのどちらかなので除外する。
df_trade = df_trade[df_trade['value'] > 0]

# 学習に不要な列を削除
df_trade.drop(columns=["txHash"], inplace=True, axis=1)


False    762627
True      56825
dtype: int64
762627


猫の属性は文字列であるため、機械学習で扱いやすいように数値型に変換する。

In [51]:
# one hot encoderを使って、猫の属性を数値に変換する
list_cols = ['body', 'coloreyes', 'eyes', 'pattern', 'mouth', 'colorprimary', 'colorsecondary', 'colortertiary', 'kitty_type']
ohe  = ce.OneHotEncoder(cols=list_cols,handle_unknown='impute')
dfOhe = ohe.fit_transform(df_trade)


## 変換表を表示する
print(pd.concat([df_trade['body'].rename('body_before'), dfOhe.filter(regex=r'body')], axis=1).head())
print(pd.concat([df_trade['coloreyes'].rename('coloreyes_before'), dfOhe.filter(regex=r'coloreyes')], axis=1).head())
print(pd.concat([df_trade['eyes'].rename('eyes_before'), dfOhe.filter(regex=r'eyes')], axis=1).head())
print(pd.concat([df_trade['pattern'].rename('pattern_before'), dfOhe.filter(regex=r'pattern')], axis=1).head())
print(pd.concat([df_trade['mouth'].rename('mouth_before'), dfOhe.filter(regex=r'mouth')], axis=1).head())
print(pd.concat([df_trade['colorprimary'].rename('colorprimary_before'), dfOhe.filter(regex=r'colorprimary')], axis=1).head())
print(pd.concat([df_trade['colorsecondary'].rename('colorsecondary_before'), dfOhe.filter(regex=r'colorsecondary')], axis=1).head())
print(pd.concat([df_trade['colortertiary'].rename('colortertiary_before'), dfOhe.filter(regex=r'colortertiary')], axis=1).head())
print(pd.concat([df_trade['kitty_type'].rename('kitty_type_before'), dfOhe.filter(regex=r'kitty_type')], axis=1).head())


  body_before  body_1  body_2  body_3  ...  body_29  body_30  body_31  body_32
0     selkirk       1       0       0  ...        0        0        0        0
4      birman       0       1       0  ...        0        0        0        0
5     ragdoll       0       0       1  ...        0        0        0        0
6      birman       0       1       0  ...        0        0        0        0
7     selkirk       1       0       0  ...        0        0        0        0

[5 rows x 33 columns]
  coloreyes_before  coloreyes_1  ...  coloreyes_31  coloreyes_32
0      thundergrey            1  ...             0             0
4             cyan            0  ...             0             0
5        mintgreen            0  ...             0             0
6         palejade            0  ...             0             0
7     coralsunrise            0  ...             0             0

[5 rows x 33 columns]
  eyes_before  coloreyes_1  coloreyes_2  ...  eyes_30  eyes_31  eyes_32
0     stunned     

scikit-learn用のデータ構造にする。  
学習データX  
教師データY

In [54]:
# 価格情報以外を学習データとする
X = dfOhe.copy()
X.drop(columns=['value'], inplace=True, axis=1)

# 取引価格を教師データにする。
Y = dfOhe['value']

# 学習用データと評価用データの分割
train_data, test_data, train_label, test_label = train_test_split(X, Y)

ランダムフォレストによる学習と評価  
出力は  
- どれくらい正解に近い値を出したかの平均値
- 正解のとの乖離の最大と最小(パーセント)

In [55]:
# ランダムフォレスト
model = RandomForestRegressor()
model.fit(train_data, train_label)

# テストデータの評価
predict = model.predict(test_data)
for i in range(len(test_label)):
  p = int(predict[i])
  t = int(test_label.iloc[i])
  #print(p, t, int(min(t, p) / max(t, p) * 100))

# 精度の平均値
rate_sum = 0
rate_max = 0
rate_min = 100
for i in range(len(test_label)):
  t = int(test_label.iloc[i])
  p = int(predict[i])
  rate_sum += int(min(t, p) / max(t, p) * 100)
  rate_max = max(rate_max, int(min(t, p) / max(t, p) * 100))
  rate_min = min(rate_min, int(min(t, p) / max(t, p) * 100))
print(rate_sum / len(test_label), rate_max, rate_min)

64.28475463782156 100 0


重回帰分析による学習と評価  
出力は  
- どれくらい正解に近い値を出したかの平均値
- 正解のとの乖離の最大と最小(パーセント)

In [56]:
# 重回帰分析
model = LinearRegression()
model.fit(train_data, train_label)

# テストデータの評価
predict = model.predict(test_data)
for i in range(len(test_label)):
  p = int(predict[i])
  t = int(test_label.iloc[i])
  #print(p, t, int(min(t, p) / max(t, p) * 100))

# 精度の平均値
rate_sum = 0
rate_max = 0
rate_min = 100
for i in range(len(test_label)):
  t = int(test_label.iloc[i])
  p = int(predict[i])
  rate_sum += int(min(t, p) / max(t, p) * 100)
  rate_max = max(rate_max, int(min(t, p) / max(t, p) * 100))
  rate_min = min(rate_min, int(min(t, p) / max(t, p) * 100))
print(rate_sum / len(test_label), rate_max, rate_min)


-119.63137625403333 99 -457674


SGD Regressor(確率的勾配降下法を用いた回帰計算)による学習と評価  
出力は  
- どれくらい正解に近い値を出したかの平均値
- 正解のとの乖離の最大と最小(パーセント)

In [57]:
# SGD Regressor
model = SGDRegressor(max_iter=1000)
model.fit(train_data, train_label)

# テストデータの評価
predict = model.predict(test_data)
for i in range(len(test_label)):
  p = int(predict[i])
  t = int(test_label.iloc[i])
  #print(p, t, int(min(t, p) / max(t, p) * 100))

# 精度の平均値
rate_sum = 0
rate_max = 0
rate_min = 100
for i in range(len(test_label)):
  t = int(test_label.iloc[i])
  p = int(predict[i])
  rate_sum += int(min(t, p) / max(t, p) * 100)
  rate_max = max(rate_max, int(min(t, p) / max(t, p) * 100))
  rate_min = min(rate_min, int(min(t, p) / max(t, p) * 100))
print(rate_sum / len(test_label), rate_max, rate_min)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


-3540158.931999968 0 -29521112960


Lassoによる学習と評価  
出力は  
- どれくらい正解に近い値を出したかの平均値
- 正解のとの乖離の最大と最小(パーセント)

In [62]:
# Lasso
model = Lasso(alpha=0.1)
model.fit(train_data, train_label)

# テストデータの評価
predict = model.predict(test_data)
for i in range(len(test_label)):
  p = int(predict[i])
  t = int(test_label.iloc[i])
  #print(p, t, int(min(t, p) / max(t, p) * 100))

# 精度の平均値
rate_sum = 0
rate_max = 0
rate_min = 100
for i in range(len(test_label)):
  t = int(test_label.iloc[i])
  p = int(predict[i])
  rate_sum += int(min(t, p) / max(t, p) * 100)
  rate_max = max(rate_max, int(min(t, p) / max(t, p) * 100))
  rate_min = min(rate_min, int(min(t, p) / max(t, p) * 100))
print(rate_sum / len(test_label), rate_max, rate_min)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.0827191864567757e+41, tolerance: 4.1971670445608055e+37
  positive)


-119.64513959502631 99 -457731


ElasticNetによる学習と評価  
出力は  
- どれくらい正解に近い値を出したかの平均値
- 正解のとの乖離の最大と最小(パーセント)

In [64]:
# ElasticNet
model = ElasticNet(random_state=0)
model.fit(train_data, train_label)

# テストデータの評価
predict = model.predict(test_data)
for i in range(len(test_label)):
  p = int(predict[i])
  t = int(test_label.iloc[i])
  #print(p, t, int(min(t, p) / max(t, p) * 100))

# 精度の平均値
rate_sum = 0
rate_max = 0
rate_min = 100
for i in range(len(test_label)):
  t = int(test_label.iloc[i])
  p = int(predict[i])
  rate_sum += int(min(t, p) / max(t, p) * 100)
  rate_max = max(rate_max, int(min(t, p) / max(t, p) * 100))
  rate_min = min(rate_min, int(min(t, p) / max(t, p) * 100))
print(rate_sum / len(test_label), rate_max, rate_min)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.0954736399999118e+41, tolerance: 4.1971670445608055e+37
  positive)


-3.3563416254994114 99 -124253


Ridge回帰による学習と評価  
出力は  
- どれくらい正解に近い値を出したかの平均値
- 正解のとの乖離の最大と最小(パーセント)

In [66]:
# Ridge
model = Ridge(alpha=1.0)
model.fit(train_data, train_label)

# テストデータの評価
predict = model.predict(test_data)
for i in range(len(test_label)):
  p = int(predict[i])
  t = int(test_label.iloc[i])
  #print(p, t, int(min(t, p) / max(t, p) * 100))

# 精度の平均値
rate_sum = 0
rate_max = 0
rate_min = 100
for i in range(len(test_label)):
  t = int(test_label.iloc[i])
  p = int(predict[i])
  rate_sum += int(min(t, p) / max(t, p) * 100)
  rate_max = max(rate_max, int(min(t, p) / max(t, p) * 100))
  rate_min = min(rate_min, int(min(t, p) / max(t, p) * 100))
print(rate_sum / len(test_label), rate_max, rate_min)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=9.88617e-19): result may not be accurate.
  overwrite_a=True).T


-119.53302321112596 99 -457038


ランダムフォレストによる学習と評価(2)  
分析に使う属性値を減らす。  
出力は  
- どれくらい正解に近い値を出したかの平均値
- 正解のとの乖離の最大と最小(パーセント)

In [67]:
# データセットの読み込み
df_trade = pd.read_csv("./data/tradeData.csv")

#重複行削除
df_trade = df_trade.drop_duplicates(keep='last')

# 欠損値の置換
df_trade = df_trade.fillna(0)

# value == 0以下を除外
# 価格が0円なのは、タダであげたか、新しく猫が生まれたかのどちらかなので除外する。
df_trade = df_trade[df_trade['value'] > 0]

# 学習に不要な列を削除
df_trade.drop(columns=["blockNumber", "tokenId", "txHash", "fur", 'base_colour', 'wild_element'], inplace=True, axis=1)

# one hot encoderを使って、猫の属性を数値に変換する
list_cols = ['body', 'coloreyes', 'eyes', 'pattern', 'mouth', 'colorprimary', 'colorsecondary', 'colortertiary', 'kitty_type']
ohe  = ce.OneHotEncoder(cols=list_cols,handle_unknown='impute')
dfOhe = ohe.fit_transform(df_trade)

# 価格情報以外を学習データとする
X = dfOhe.copy()
X.drop(columns=['value'], inplace=True, axis=1)

# 取引価格を教師データにする。
Y = dfOhe['value']

# 学習用データと評価用データの分割
train_data, test_data, train_label, test_label = train_test_split(X, Y)

# ランダムフォレスト
model = RandomForestRegressor()
model.fit(train_data, train_label)

# テストデータの評価
predict = model.predict(test_data)
for i in range(len(test_label)):
  p = int(predict[i])
  t = int(test_label.iloc[i])
  #print(p, t, int(min(t, p) / max(t, p) * 100))

# 精度の平均値
rate_sum = 0
rate_max = 0
rate_min = 100
for i in range(len(test_label)):
  t = int(test_label.iloc[i])
  p = int(predict[i])
  rate_sum += int(min(t, p) / max(t, p) * 100)
  rate_max = max(rate_max, int(min(t, p) / max(t, p) * 100))
  rate_min = min(rate_min, int(min(t, p) / max(t, p) * 100))
print(rate_sum / len(test_label), rate_max, rate_min)

48.282320632200936 99 0


ランダムフォレストによる学習と評価(3)  
レア猫は属性を持たないが価値のある猫のため、予測モデルに変な影響を与えている可能性がある。  
そのため、予測対象から除外する。  
出力は  
- どれくらい正解に近い値を出したかの平均値
- 正解のとの乖離の最大と最小(パーセント)

In [ ]:
# データセットの読み込み
df_trade = pd.read_csv("./data/tradeData.csv")

#重複行削除
df_trade = df_trade.drop_duplicates(keep='last')

# 欠損値の置換
df_trade = df_trade.fillna(0)

# value == 0以下を除外
# 価格が0円なのは、タダであげたか、新しく猫が生まれたかのどちらかなので除外する。
df_trade = df_trade[df_trade['value'] > 0]

# 学習に不要な列を削除
df_trade.drop(columns=["txHash", "kitty_type"], inplace=True, axis=1)

# one hot encoderを使って、猫の属性を数値に変換する
list_cols = ['body', 'coloreyes', 'eyes', 'pattern', 'mouth', 'colorprimary', 'colorsecondary', 'colortertiary']
ohe  = ce.OneHotEncoder(cols=list_cols,handle_unknown='impute')
dfOhe = ohe.fit_transform(df_trade)

# 価格情報以外を学習データとする
X = dfOhe.copy()
X.drop(columns=['value'], inplace=True, axis=1)

# 取引価格を教師データにする。
Y = dfOhe['value']

# 学習用データと評価用データの分割
train_data, test_data, train_label, test_label = train_test_split(X, Y)

# ランダムフォレスト
model = RandomForestRegressor()
model.fit(train_data, train_label)

# テストデータの評価
predict = model.predict(test_data)
for i in range(len(test_label)):
  p = int(predict[i])
  t = int(test_label.iloc[i])
  #print(p, t, int(min(t, p) / max(t, p) * 100))

# 精度の平均値
rate_sum = 0
rate_max = 0
rate_min = 100
for i in range(len(test_label)):
  t = int(test_label.iloc[i])
  p = int(predict[i])
  rate_sum += int(min(t, p) / max(t, p) * 100)
  rate_max = max(rate_max, int(min(t, p) / max(t, p) * 100))
  rate_min = min(rate_min, int(min(t, p) / max(t, p) * 100))
print(rate_sum / len(test_label), rate_max, rate_min)